# **Ataque por Compresión**

Instalamos las instancias necesarias para correr el codigo. 

Siendo **DEFLATE** la libreria usada para la parte de compresión.

Y **pycrypto** necesaria para la encripción con **AES** 

In [99]:
!pip install deflate
!pip3 install pycrypto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Encriptamos con **AES** usando la libreria anteriormente mensionada, esta vez diferente al codigo usado para general la grafica, usamos **AES** en modo **CTR**, siendo esta la forma de cifrar en modo de flujo permitiendo mostrar la idea del ataque más clara sin necesidad de introducirnos al añadido de padding para separar los bloques

In [100]:
# AES 256 encryption/decryption using pycrypto library
 
import base64
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util import Counter
from Crypto.Protocol.KDF import PBKDF2
import os
 
BLOCK_SIZE = 16
pad = lambda s: s + bytes((BLOCK_SIZE - len(s) % BLOCK_SIZE) * " ", encoding = "utf-8")
unpad = lambda s: s[:-ord(s[len(s) - 1:])]
password = "contrasenaSegura"

#seed
counter = os.urandom(16)

def get_private_key(password):
    salt = b"Esto es la sal"
    kdf = PBKDF2(password, salt, 64, 1000)
    key = kdf[:32]
    return key
 
def encrypt(raw, password):
    private_key = get_private_key(password)
    raw = pad(raw)
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(private_key, AES.MODE_CTR, counter=lambda: counter)
    return base64.b64encode(cipher.encrypt(raw))

print(encrypt(bytes("Mensaje de prueba", encoding = "utf-8" ), password))

b'0DLVQ7QzZeKlDqi24izO+vx3mxD1eSDi4Uuo5rB5i7g='


Definimos una lista llamada *abecedario* que adicional a contener las letras en mayúsculas y minúsculas contiene números y caracteres que son comúnmente permitidos en estos sistemas

In [101]:
import string
abecedario = list(dict.fromkeys(string.ascii_lowercase, 0).keys())
abecedario.extend(list(dict.fromkeys(string.ascii_uppercase, 0).keys()))
abecedario.extend([i for i in "0123456789_.-+/=:;"])

print(len(abecedario),abecedario)

70 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', '.', '-', '+', '/', '=', ':', ';']


Definimos *aut_cookie_2* que representa a la *cookie* en sí que explotara el atacante y teniendo en cuenta la idea del ataque y que el atacante sabe de comienzo como se ve el comienzo de la *cookie* definimos *know*.

Simulando lo que haria el protocolo diremos que cada bloque pesa 205, así que itero añadiendole padding a una cadena cortada del *aut_cookie_2* hasta llegar a ese peso. *(Recuerde que esto se hace para fines practicos de entender el ataque)* Llegando a que el padding necesario es de 18 bytes aleatorios.

In [108]:
import sys
import deflate

aut_cookie_2 = """
POST /echo/post/json HTTP/1.1
Host: bancopolombia.com
Cookie SECRET: 7xc89f+94qwrtraqueta
"""

know = """
POST /echo/post/json HTTP/1.1
Host: bancopolombia.com
Cookie SECRET: """


cookie_base = bytes(aut_cookie_2, encoding = "utf-8") # Pesa 185
#Decimos que el peso del bloque es de 205

#Tomo una cadena cortada de el aut_cookie_2 simulando que es el corte del bloque como base
cookie_base = bytes(aut_cookie_2[:-20], encoding = "utf-8") # Peso cookie cortada = 161
peso_envio = sys.getsizeof(encrypt(deflate.gzip_compress(cookie_base, 9), password))
n = 0
while peso_envio < 205:
  n += 1                                                        #Añadimos n-bytes aleatorios con el fin de llegar al peso deseado
  cookie_base = bytes(know + aut_cookie_2[:-20] , encoding = "utf-8") + Random.get_random_bytes(n) 
  peso_envio = sys.getsizeof(encrypt(deflate.gzip_compress(cookie_base, 9), password))

  print(peso_envio)

print(n)

161
161
185
185
185
185
185
185
185
185
185
185
185
185
185
185
185
205
18


# Inicio del Ataque

Teniendo que el padding ya estipulado *(18)* notemos que se le tiene que quitar un bit al padding para añadir la **adivinanza** = **letter**.

Almacenamos el peso de cada petición en *aux* y observamos cuando dicho peso baja, dado que si este es menor a los otros significa que hay una mejor compresión y por lo tanto la **adivinanza** es correcta.

In [103]:

aux = {}
min = None 
padding = Random.get_random_bytes(17)

for letter in abecedario:
  aux_bloque = padding + bytes( know + letter + aut_cookie_2[:-20], encoding = "utf-8") 
  peso_aux = sys.getsizeof(encrypt(deflate.gzip_compress(aux_bloque,9), password))

  #guardamos el peso de cada peticion junto con la letra respectiva
  aux[letter] = peso_aux

  #actualizamos el peso más bajo
  if min == None:
      min = peso_aux
  elif peso_aux < min:
      min = peso_aux

#filtro solo los que tengan el peso minimo encontrado
cant_min = dict(filter(lambda x: x[1] == min, aux.items()))

print(len(cant_min),cant_min) 

2 {'b': 185, '7': 185}


Nos lanza 2 letras que tiene el minimo peso registrado. 

> Si se cambia la cantidad de padding a un byte más o uno menos los pesos para las 70 letras es el mismo por lo que no se nota una vulnerabilidad



Comenzos a probar con estos 2 resultados:

Note que se le vuelve a quitar un byte al padding otra vez y que el aut_cookie_2 se le añade una letra. Esto con el fin de que a la hora de comprimir estos 2 coinsidan y que en los demas casos falle mostrando un peso mayor.

In [104]:
abecedario_aux_2 = {'b': 185, '7': 185}
aux = {}
min = None 
padding = Random.get_random_bytes(16)

for key in abecedario_aux_2.keys():
  for letter in abecedario:
    aux_bloque = bytes( know + key + letter + aut_cookie_2[:-19], encoding = "utf-8") + padding
    peso_aux = sys.getsizeof(encrypt(deflate.gzip_compress(aux_bloque, 1), password))

    aux[letter] = peso_aux
    if min == None:
        min = peso_aux
    elif peso_aux < min:
        min = peso_aux

  cant_min = dict(filter(lambda x: x[1] == min, aux.items()))

  print(len(cant_min),cant_min) 

1 {'a': 185}
1 {'x': 185}


Y ahora Semi-automatizamos el proceso

# Ataque TLS Semi-Automatizado

Realizamos lo anterior de forma iterativa tomando como base 17 bytes aleatorios de padding y reduciendo su cantidad en cada iteración para introducir una nueva *adivinanza*.

In [118]:
aut_cookie_2 = """
POST /echo/post/json HTTP/1.1
Host: bancopolombia.com
Cookie SECRET: 7xc89f+94qwrtraqueta
"""

know = """
POST /echo/post/json HTTP/1.1
Host: bancopolombia.com
Cookie SECRET: """


aux = {}
min = None 
n = 0
while n != 10:
  padding = Random.get_random_bytes(17-n)
  for letter in abecedario:
    aux_bloque = bytes( know + letter + aut_cookie_2[:-20+n], encoding = "utf-8") + padding
    peso_aux = sys.getsizeof(encrypt(deflate.gzip_compress(aux_bloque, 9), password))

    aux[letter] = peso_aux
    if min == None:
        min = peso_aux
    elif peso_aux < min:
        min = peso_aux

  cant_min = dict(filter(lambda x: x[1] == min, aux.items()))
  print("cant_min" , len(cant_min),cant_min) 

  if len(cant_min) > 1:
    know += input("cual le añadimos - tenga encuenta a 'cant_min' : ")
  else:
    for i in cant_min.keys():
      know += i
  
  n += 1
  print(" ")
print(know)

cant_min 2 {'b': 185, '7': 185}
cual le añadimos - tenga encuenta a 'cant_min' : 7
 
cant_min 1 {'x': 185}
 
cant_min 1 {'c': 185}
 
cant_min 1 {'8': 185}
 
cant_min 1 {'9': 185}
 
cant_min 1 {'f': 185}
 
cant_min 1 {'+': 185}
 
cant_min 1 {'9': 185}
 
cant_min 1 {'4': 185}
 
cant_min 1 {'q': 185}
 

POST /echo/post/json HTTP/1.1
Host: bancopolombia.com
Cookie SECRET: 7xc89f+94q
